# Facial Expression Classifier

In [6]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/



In [7]:

!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

Dataset URL: https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset
License(s): unknown
face-expression-recognition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [10]:
#generating dataset ,using keras.utils.image_dataset_from_directory,from my folders

train_dataset=keras.utils.image_dataset_from_directory(
    directory='./images/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)


)
print("Label names:", train_dataset.class_names)


Found 28821 files belonging to 7 classes.
Label names: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [11]:
#creating validation dataset

In [12]:
validation_dataset=keras.utils.image_dataset_from_directory(
directory='./images/validation/',
labels='inferred',
label_mode='int',
batch_size=32,
image_size=(256,256)

)

Found 7066 files belonging to 7 classes.


In [13]:
#normalizing the data :
#from 0-255 to 0-1

In [14]:
def process(image,label):
    image=tf.cast(image/255. ,tf.float32)
    return image,label
train_dataset=train_dataset.map(process)



In [15]:
validation_dataset=validation_dataset.map(process)


In [16]:
from tensorflow.keras.layers import Input
from tensorflow.keras import regularizers

In [17]:
model=Sequential()
model.add(Input(shape=(256, 256, 3)))
model.add(Conv2D(32,kernel_size=(3,3),padding="same",activation="relu",input_shape=(256,256,3)))
#here 32,kernel size is ig default ,lets change them later,padding valid ,means
#not focusing on edges .but more focus on center as no padding,activation =
#relu ->good for non linear data
#256,256, 3->RGB channels
model.add(BatchNormalization()) # helps in avoiding local minima ,speed up,model more stable
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))

#2ndconv layer
model.add(Conv2D(64,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))

#3rd layer
model.add(Conv2D(128,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))

#4th layer
model.add(Conv2D(256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))

model.add(Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="same"))


#flattening for ann
model.add(Flatten()) #2D -> 1D ,for ann

model.add(Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(7,activation="softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

#using sparse categorical crossentropy as its goo for multi class classification

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,995,207 (38.13 MB)

 Trainable params: 9,993,223 (38.12 MB)

 Non-trainable params: 1,984 (7.75 KB)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop=EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=1,
    restore_best_weights=True

)

In [21]:
history=model.fit(train_dataset,epochs=25,callbacks=[early_stop],validation_data=validation_dataset)

Epoch 1/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 114s 109ms/step - accuracy: 0.2193 - loss: 6.6235 - val_accuracy: 0.2583 - val_loss: 1.8184
Epoch 2/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 113s 89ms/step - accuracy: 0.2459 - loss: 1.8187 - val_accuracy: 0.2583 - val_loss: 1.8091
Epoch 3/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 79s 85ms/step - accuracy: 0.2457 - loss: 1.8146 - val_accuracy: 0.2583 - val_loss: 1.8092
Epoch 4/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 86s 90ms/step - accuracy: 0.2457 - loss: 1.8152 - val_accuracy: 0.2583 - val_loss: 1.8092
Epoch 5/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 77s 85ms/step - accuracy: 0.2465 - loss: 1.8141 - val_accuracy: 0.2583 - val_loss: 1.8095
Epoch 6/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 77s 85ms/step - accuracy: 0.2459 - loss: 1.8152 - val_accuracy: 0.2583 - val_loss: 1.8092
Epoch 7/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 86s 90ms/step - accuracy: 0.2458 - loss: 1.8144 - val_accuracy: 0.2583 - val_loss: 1.8093
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 2.


In [ ]:
# too bad accuracy -> but by future projects i will learn to increase it.